In [389]:
import json, random, time, os
import numpy as np
from pprint import pprint
from collections import Counter, defaultdict
from bs4 import BeautifulSoup
import requests
import urllib.request
import re
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import pickle
import copy
import string
import wikipedia
import spacy
from itertools import tee
import pylcs
np.set_printoptions(precision=4)

In [2]:
API_URL = 'http://en.wikipedia.org/w/api.php'

In [200]:
nlp = spacy.load('en_core_web_sm')

In [3]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]

In [134]:
url_blocklist = ['seal', 'sign ', 'pdf', 'gif', 'icon', 'notice', 'cartoon', 'publish', 'menu', 'logo', 'svg', 'webm', 'page', \
                     'ogg', 'flickr', 'poster', 'ogv', 'banner', 'tif', 'montage', 'centralautologin', 'footer']

In [202]:
pos_list = ['NUM', 'NOUN', 'ADJ', 'PROPN']

In [204]:
pattern = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*|\(|\)|-')

In [207]:
PUNCTUATIONS = set(string.punctuation)

In [191]:
new_txt_data = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/output_mine_all_schema.json", "r"))
print(len(new_txt_data))

7921


In [216]:
def IoU(A, B):
    intersection = len(A.intersection(B))
    union = len(A.union(B))
    return round(intersection / (union+1e-7), 2)

In [101]:
def _wiki_request(params):
  
    global USER_AGENT_LIST

    params['format'] = 'json'
    if not 'action' in params:
        params['action'] = 'query'

    headers = {
        'User-Agent': random.choice(USER_AGENT_LIST)#'wikipedia (https://github.com/goldsmith/Wikipedia/)'
    }

    r = requests.get(API_URL, params=params, headers=headers)
    return r.json()

In [97]:
def content(title):

    query_params = {
        'prop': 'extracts|revisions',
        'explaintext': '',
        'rvprop': 'ids',
        'titles': title
    }
    request = _wiki_request(query_params)
    result = request['query']['pages']
    content = result[list(result.keys())[0]]['extract']
    return content

In [105]:
### Takeaway: get html from wikipedia pypi is somehow pretty slow. But the library gives cleaner content
### Decision: html is to be obtained from url request; 
###           content is to be obtained from the (stolen) library function, with USER_AGENT_LIST (using the wiki github user agent doesn't make much difference)

start = time.time()
for i in range(100):
    if i%10 == 9: print(i)
    url = "https://en.wikipedia.org/wiki/Egyptian_tombs"
    req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    with urllib.request.urlopen(req) as f:
        html = f.read().decode('utf-8')
    end_indx = html.find('<h2><span class="mw-headline" id="References">References</span>')
    html = html[:end_indx]
    soup = BeautifulSoup(html, 'html.parser')
    cont = content("Egyptian tombs") # The stolen function works faster than wikipedia.page().content
print(time.time() - start)

9
19
29
39
49
59
69
79
89
99
40.43351101875305


In [170]:
content('Elfed')

"Elfed is a Welsh language place name and personal name. It commonly refers to one of two geographic areas:\n\nThe Kingdom of Elfed (English: Elmet), an Old Welsh kingdom in what is now northern England during the Early Middle Ages\nElfed, a cymwd (commote) of Cantref Gwarthaf (in modern Carmarthenshire, Wales), later anglicised to 'Elvet Hundred'Elfed may also refers to:\n\nCynwyl Elfed, a community in the cymwd of Elfed, Wales\nElfed High School, in Buckley, Flintshire, Wales\nElfed Davies, Baron Davies of Penrhys (1913–1992), Welsh politician\nElfed Evans (1926–1988), Welsh professional footballer\nHowell Elvet Lewis hymn-writer, poet and Archdruid whose bardic name was Elfed\nElfed Morris (born 1942), Welsh professional footballer"

In [140]:
def get_html(url):
    req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    with urllib.request.urlopen(req) as f:
        html = f.read().decode('utf-8')
    end_indx = html.find('<h2><span class="mw-headline" id="References">References</span>')
    html = html[:end_indx]
    return html

In [259]:
### Scrap imgs and their captions. 
def get_imgs_and_captions(html):
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('img')
    imgUrls = []
    captions = []
    for l in links:
        imgUrl = l.get('src')
        try: 
            width = int(l['width'])
            height = int(l['height'])
        except:
            continue
        if width<100 or height<100:
            continue
        if any(b in imgUrl.lower() for b in url_blocklist): 
            continue
        imgUrls.append(imgUrl)

        # Special case for thumb images, which are located inside a table
        thumbinner_div = l.find_parent("div", class_='thumbinner')
        if thumbinner_div: 
            captions.append(thumbinner_div.text)
            continue
        
        segments = []
        prev_th = l.find_previous('th')
        if prev_th: segments.append(prev_th.get_text(strip=True))
            
        tr_parent = l.find_parent('tr')
        if tr_parent: segments.append(tr_parent.get_text(strip=True))

        captions.append(". ".join(segments))
    return imgUrls, captions

In [234]:
url = 'https://en.wikipedia.org/wiki/Beirut'
html = get_html(url)
imgs, caps = get_imgs_and_captions(html)
for im, cap in zip(imgs, caps):
    print(im)
    print(cap)
    print(" ----------------------- ")

don't have width or height:  //upload.wikimedia.org/wikipedia/commons/thumb/a/ab/Beirut_1965.webm/120px--Beirut_1965.webm.jpg
don't have width or height:  //upload.wikimedia.org/wikipedia/commons/thumb/2/29/Beirut-in-1919.webm/120px--Beirut-in-1919.webm.jpg
//upload.wikimedia.org/wikipedia/commons/thumb/4/4d/ChurchMosque.jpg/124px-ChurchMosque.jpg
BeirutبيروتBeyrouth. Top to bottom, and left to right:Maronite Cathedral of Saint George(left) andMohammad Al-Amin Mosque(right), Clock Tower atNejmeh Square,Sahat al Shouhada,Sursock Museum, Pigeon Rocks ofRaouché
 ----------------------- 
//upload.wikimedia.org/wikipedia/commons/thumb/a/a5/1934_Clock_Tower.PNG/124px-1934_Clock_Tower.PNG
BeirutبيروتBeyrouth. Top to bottom, and left to right:Maronite Cathedral of Saint George(left) andMohammad Al-Amin Mosque(right), Clock Tower atNejmeh Square,Sahat al Shouhada,Sursock Museum, Pigeon Rocks ofRaouché
 ----------------------- 
//upload.wikimedia.org/wikipedia/commons/thumb/4/47/Lebanon_location

In [211]:
def find_pages_by_hyperlink(keywords, url):
    if 'en.wikipedia.org' not in url: return {}
    anchor2page = {}
    req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    with urllib.request.urlopen(req) as f:
        html = f.read().decode('utf-8')
    end_indx = html.find('<h2><span class="mw-headline" id="References">References</span>')
    html = html[:end_indx]
    soup = BeautifulSoup(html, 'html.parser')
    links = soup.find_all('a', attrs={'href': re.compile("^/wiki/(?!.*(:|\(identifier\))).*")})
    for link in links:
        title = link.get('title')
        text = link.text
        if title is None or not text: continue
        #print(link)
        if any(b in keywords for b in title.split()):
            pagelink = 'https://en.wikipedia.org' + link.get('href')
            if pagelink.find("#") > -1:
                pagelink = pagelink[:pagelink.find("#")]
            anchor2page[__load(title)] = pagelink
        if len(text) == 0: print(link)
        elif pylcs.lcs(title.lower(), text.lower())/len(text) < 0.85:
            if any(b in keywords for b in text.split()):
                pagelink = 'https://en.wikipedia.org' + link.get('href')
                if pagelink.find("#") > -1:
                    pagelink = pagelink[:pagelink.find("#")]
                anchor2page[__load(title)] = pagelink
    if '' in anchor2page: del anchor2page['']
    return anchor2page

In [341]:
def get_keywords_and_relevant_pages_from_txt_sample(k):
    Q = new_txt_data[str(k)]['Q']
    doc = nlp(Q)
    keywords = set([t.text for s in doc.sents for t in s if t.pos_ in pos_list or ((not t.is_sent_start) and t.text[0].isupper())])
    titlewords = set()
    anchor2page = {}
    for f in new_txt_data[str(k)]['SupportingFacts']:
        if not 'en.wikipedia.org/wiki/' in f['url']: continue
        fact_title_raw = ' '.join(urllib.parse.unquote(f['url']).split('/')[-1].split('_'))
        fact_title = pattern.sub('', fact_title_raw)
        titlewords = titlewords.union(fact_title.split())
        for title in wikipedia.search(fact_title_raw):
            anchor2page[__load(title)] = "https://en.wikipedia.org/wiki/" + urllib.parse.quote("_".join(title.split()))
    keywords = keywords - PUNCTUATIONS
    keywords = set(sum([[w.capitalize(), w.lower()] for w in keywords], []))
    titlewords = titlewords - PUNCTUATIONS
    
    print("#pages before extension by hyperlink: ", len(anchor2page))
    for f in new_txt_data[str(k)]['SupportingFacts']:
        d = find_pages_by_hyperlink(keywords.union(titlewords), f['url'])
        anchor2page.update(d)
    if '' in anchor2page: del anchor2page['']
    print("#pages after extension by hyperlink: ", len(anchor2page))
    
    A = new_txt_data[str(k)]['A']
    doc = nlp(A)
    answerwords = set([t.text for t in doc if t.pos_ in pos_list or ((not t.is_sent_start) and t.text[0].isupper())]) - keywords
    answerwords = answerwords - PUNCTUATIONS
    answerwords = set(sum([[w.capitalize(), w.lower()] for w in answerwords], []))
    goldfactwords = set()
    Q_A_words = keywords.union(answerwords)
    for f in new_txt_data[str(k)]['SupportingFacts']:
        doc = nlp(f['fact'])
        goldfactwords = goldfactwords.union(set([t.text for t in doc if t.pos_ in pos_list or ((not t.is_sent_start) and t.text[0].isupper())]) - Q_A_words)
    goldfactwords = goldfactwords - PUNCTUATIONS
    goldfactwords = set(sum([[w.capitalize(), w.lower()] for w in goldfactwords], []))
    
    for a in list(anchor2page.keys()):
        if is_disambiguation_page(anchor2page[a]):
            print(a, " is an disambiguation page")
            for t in recover_disambiguation_page(a):
                anchor2page[__load(t)] = "https://en.wikipedia.org/wiki/" + urllib.parse.quote("_".join(t.split()))
            del anchor2page[a]
    if '' in anchor2page: del anchor2page['']
    return titlewords, keywords, goldfactwords, answerwords, Q, A, anchor2page

In [171]:
def get_page_categories(title):
    url = 'https://en.wikipedia.org/w/api.php?format=xml&action=query&prop=categories&titles='+urllib.parse.quote(title)
    req = urllib.request.Request(url, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
    with urllib.request.urlopen(req) as f:
        xml = f.read().decode('utf-8')
    soup= BeautifulSoup(xml,"lxml-xml")
    tags = soup.find('categories')
    if tags is None: return []
    categories = [c.get('title').replace("Category:", "") for c in tags]
    return categories
def is_disambiguation_page(url):
    title = url.split('/')[-1]
    return 'disambiguation' in " ".join(get_page_categories(title))
def recover_disambiguation_page(title):
    try: 
        wikipedia.page(title)
        return []
    except Exception as e:
        err = str(e)
        return err[err.find("may refer to:")+13:].strip().split('\n')

In [179]:
def __load(title):
    '''
    Load basic information from Wikipedia.
    Confirm that page exists and is not a disambiguation/redirect.
    Does not need to be called manually, should be called automatically during __init__.
    '''
    query_params = {
        'prop': 'info|pageprops',
        'inprop': 'url',
        'ppprop': 'disambiguation',
        'redirects': '',
        'titles': title
    }

    request = _wiki_request(query_params)

    query = request['query']
    pageid = list(query['pages'].keys())[0]
    page = query['pages'][pageid]

    # missing is present if the page is missing
    if 'missing' in page:
        print("Page is missing: ", title)
        return ""

    # same thing for redirect, except it shows up in query instead of page for whatever silly reason
    elif 'redirects' in query:
        redirects = query['redirects'][0]
        return redirects['to']

    return title

In [350]:
def find_sentences_from_page(title, page, keywords, answerwords, goldfactwords):
    try: 
        cont = content(title)
        paragraphs = cont[:cont.find('== References ==')].split('\n')
        
    except: 
        print("Exception from find_sentences_from_page, title = ", title)
        return {}
    sen2score = {}
    for p in paragraphs:
        if len(p.split()) >= 10:
            doc = nlp(p)
            for s in doc.sents:
                if len(s) < 10: 
                    continue
                nouns_in_s = [t.text for t in s if (t.pos_ in pos_list or ((not t.is_sent_start) and t.text[0].isupper()))]

                IoU_Q = IoU(set(nouns_in_s), keywords)
                IoU_A = IoU(set(nouns_in_s), answerwords)
                if IoU_Q -  IoU_A > 0.06:
                    #print(IoU_Q, IoU_A, s.text)
                    IoU_G = IoU(set(nouns_in_s), goldfactwords)
                    sen2score[s.text] = {'scores': (IoU_Q, IoU_A, IoU_G, IoU_Q - IoU_A, IoU_Q - IoU_A - IoU_G), 'link': page, 'title': title}

    for p in paragraphs:
        if len(p.split()) >= 10:
            doc = nlp(p)
            it1, it2 = tee(doc.sents)
            next(it2, None)
            for s1, s2 in zip(it1, it2):
                if len(s1) < 5 or len(s2) < 5 or len(s1)+len(s2) > 70 or len(s1)+len(s2) < 10: 
                    continue 
                nouns_in_s = [t.text for s in [s1, s2] for t in s if (t.pos_ in pos_list or ((not t.is_sent_start) and t.text[0].isupper()))]

                IoU_Q = IoU(set(nouns_in_s), keywords)
                IoU_A = IoU(set(nouns_in_s), answerwords)
                if IoU_Q -  IoU_A >= 0.06:
                    #print(IoU_Q, IoU_A, " ".join([s1.text, s2.text]))
                    IoU_G = IoU(set(nouns_in_s), goldfactwords)
                    sen2score[" ".join([s1.text, s2.text])] = {'scores': (IoU_Q, IoU_A, IoU_G, IoU_Q - IoU_A, IoU_Q - IoU_A - IoU_G), 'link': page, 'title': title}
                    #print(s)
    #print(len(sen2score))
    return sen2score

In [351]:
def find_imgs_from_page(title, page, keywords, answerwords, goldfactwords):
    try: 
        html = get_html(page)
        imgs, caps = get_imgs_and_captions(html)
        
    except: 
        print("Exception from find_imgs_from_page, page = ", page)
        return {}
    
    cap2score = {}
    for im, cap in zip(imgs, caps):
        doc = nlp(cap)
        nouns_in_s = [t.text for t in doc if (t.pos_ in pos_list or ((not t.is_sent_start) and t.text[0].isupper()))]

        IoU_Q = IoU(set(nouns_in_s), keywords)
        IoU_A = IoU(set(nouns_in_s), answerwords)
        if IoU_Q -  IoU_A > 0.06:
            IoU_G = IoU(set(nouns_in_s), goldfactwords)
            cap2score[doc.text] = {'scores': (IoU_Q, IoU_A, IoU_G, IoU_Q - IoU_A, IoU_Q - IoU_A - IoU_G), 'img':im, 'link': page, 'title': title}    
    #print(len(cap2score))
    return cap2score

In [352]:
def get_sen2score_from_indx(k):
    print('k = ', k)
    titlewords, keywords, goldfactwords, answerwords, Q, A, anchor2page = get_keywords_and_relevant_pages_from_txt_sample(k)
    print("Q = ", Q)
    print("A = ", A)
    print("keywords = ", keywords)
    print("titlewords = ", titlewords)
    print("answerwords = ", answerwords)
    print("goldfactwords = ", goldfactwords)
    
    print(" -- find sentences from each page --")
    sen2score = {}
    cap2score = {}
    for title in anchor2page:
        sen2score.update(find_sentences_from_page(title, anchor2page[title], keywords, answerwords, goldfactwords))
        cap2score.update(find_imgs_from_page(title, anchor2page[title], keywords, answerwords, goldfactwords))
    sen2score = dict(sorted(sen2score.items(), key=lambda x: x[1]['scores'][-1], reverse=True))
    cap2score = dict(sorted(cap2score.items(), key=lambda x: x[1]['scores'][-1], reverse=True))
    print("total num of sentences found = ", len(sen2score))
    print("total num of imgs found = ", len(cap2score))
    
    word_lists = (titlewords, keywords, goldfactwords, answerwords)
    return sen2score, cap2score, word_lists

In [333]:
def highlight_words(word_lists, colors, sentence):
    s = copy.deepcopy(sentence)
    for word_list, color in zip(word_lists, colors):
        try: 
            if "".join(word_list): 
                s = re.sub(r'\b(' + r'|'.join([re.escape(c) for c in word_list]) + r')\b\s*', lambda m: '<span style="background-color:rgb{}">{}</span>'.format(color, m.group()), s)
        except: 
            print(s)
            print(word_list)
            raise
    return s

In [357]:
def add_html_row(k, sen2score, cap2score, word_lists, colors = ["(223, 255, 238)", "(193, 239, 253)", "(253, 252, 152)", "(255, 214, 222)"]):
    html = ""
    html += '<tr><td>{}.</td><td>Q: {}<br>'.format(k, highlight_words(word_lists, colors, new_txt_data[str(k)]['Q']))
    for f in new_txt_data[str(k)]['SupportingFacts']:
        html += '<br><br>&nbsp;&nbsp;{}'.format(highlight_words(word_lists, colors, f['fact']))
        html += '<a href="{}"> link</a>'.format(f['url'])
    html += '<br><br>A: {}<br>'.format(highlight_words(word_lists, colors, new_txt_data[str(k)]['A']))
    html += '</td><td>'
    s_buckets = defaultdict(lambda: [])
    for s in sen2score:
        if sen2score[s]['scores'][2] == 0.0 and sen2score[s]['scores'][1] == 0.0 and len(s.split()) in range(22, 60):
            s_buckets['good'].append(s)
        elif sen2score[s]['scores'][1] > 0.0 or sen2score[s]['scores'][2] > 0.0:
            s_buckets['(maybe)falseneg'].append(s)
    
    for s in s_buckets['good'][:10]:
        html += '{} --- {} '.format(highlight_words(word_lists, colors, s), str(sen2score[s]['scores']))
        html += '<a href="{}"> {}</a><br><br>'.format(sen2score[s]['link'], sen2score[s]['title'])
    html += '<strong> ----------------- (maybe)falseneg ---------------- </strong><br>'
    for s in random.sample(s_buckets['(maybe)falseneg'], min(len(s_buckets['(maybe)falseneg']), 5)):
        html += '{} --- {} '.format(highlight_words(word_lists, colors, s), str(sen2score[s]['scores']))
        html += '<a href="{}"> {}</a><br><br>'.format(sen2score[s]['link'], sen2score[s]['title'])
        
    html += '</td><td>'
    for k in list(cap2score.keys())[:10]:
        html += '<a href="{}" class="tool-tip" target="_blank"><img style="display:block; max-height:300px; max-width:100%;" src = "{}"></a>'.format(cap2score[k]['img'], cap2score[k]['img'])
        html += '<br>Caption: {}<br>{}<br>'.format(highlight_words(word_lists, colors, k), str(cap2score[k]['scores']))
        html += '<a href="{}"> {}</a><br><br>'.format(cap2score[k]['link'], cap2score[k]['title'])
    html += '</td></tr>'
    html += '<tr><td colspan=4><hr></td></tr>'
    return html.encode('ascii', 'xmlcharrefreplace').decode("utf-8")

In [384]:
### Mining + Save as json
upd_txt_data = {} #json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data.json", "r"))
for k in [3787]:
    if str(k) in upd_txt_data: continue
    upd_txt_data[str(k)] = copy.deepcopy(new_txt_data[str(k)])
    upd_txt_data[str(k)]['new_negFacts'] = []
    upd_txt_data[str(k)]['img_negFacts'] = []
    sen2score, cap2score, word_lists = get_sen2score_from_indx(k)
    upd_txt_data[str(k)]['word_lists'] = {
        'titlewords': " || ".join(word_lists[0]), 
        'keywords': " || ".join(word_lists[1]), 
        'goldfactwords': " || ".join(word_lists[2]), 
        'answerwords': " || ".join(word_lists[3])
    }
    new_negFacts_count = 0
    for s in sen2score:
        if new_negFacts_count >= 40: break
        if sen2score[s]['scores'][2] == 0.0 and sen2score[s]['scores'][1] == 0.0 and len(s.split()) in range(22, 60):
            upd_txt_data[str(k)]['new_negFacts'].append({
                'title': sen2score[s]['title'],
                'scores': str(sen2score[s]['scores']),
                'fact': s,
                'url': sen2score[s]['link']
            })
            new_negFacts_count += 1
    
    img_negFacts_count = 0
    for c in cap2score:
        if img_negFacts_count >= 40: break
        upd_txt_data[str(k)]['img_negFacts'].append({
            'title': cap2score[c]['title'],
            'scores': str(cap2score[c]['scores']),
            'caption': c,
            'url': cap2score[c]['link'],
            'imgUrl': cap2score[c]['img']
        })
        img_negFacts_count += 1
json.dump(upd_txt_data, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data.json", "w"), indent=4)

k =  3787
#pages before extension by hyperlink:  18
#pages after extension by hyperlink:  42
Q =  Which rival groups did the people who are currently the dominant ethnic group in the Xinjiang region compete with in the past?
A =  They competed with other Altaic tribes, Indo-European empires from the south and west and Sino-Tibetan empires to the east.
keywords =  {'dominant', 'ethnic', 'rival', 'Region', 'region', 'xinjiang', 'Past', 'Rival', 'Xinjiang', 'groups', 'group', 'Group', 'People', 'people', 'Dominant', 'Groups', 'Ethnic', 'past'}
titlewords =  {'History', 'Xinjiang', 'Christianity', 'people', 'Uyghur'}
answerwords =  {'tribes', 'indo', 'south', 'West', 'Sino', 'Tibetan', 'european', 'Other', 'Empires', 'Altaic', 'other', 'Indo', 'South', 'tibetan', 'east', 'European', 'East', 'empires', 'sino', 'Tribes', 'west', 'altaic'}
goldfactwords =  {'Brief', 'Republic', 'powers', 'story', 'Christianity', 'muslim', 'Tribe', 'few', 'Mountains', 'Nomadic', 'nomadic', 'small', 'Story', 'r

In [417]:
### check progress
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data/"

if os.path.isdir(path):
    data = {}
    files = os.listdir(path)
    print("{} files found".format(len(files)))
    for f in files:
        if not '.json' in f: continue
        data.update(json.load(open(os.path.join(path, f), "r")))
else:
    data = json.load(open(path, "r"))
print("{} samples found".format(len(data)))

9 files found
657 samples found


In [410]:
print(max([k for k in data if int(k)>3000 and int(k)<4000]))
print(max([k for k in data if int(k)>4000 and int(k)<5000]))
print(max([k for k in data if int(k)>6000 and int(k)<7000]))

3053
4041
6051


In [394]:
### Create demo from json (dataset is divided into 8 chunks of size 1000 for parallel distractor mining)
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data/"

if os.path.isdir(path):
    data = {}
    files = os.listdir(path)
    print("{} files found".format(len(files)))
    for f in files:
        if not '.json' in f: continue
        data.update(json.load(open(os.path.join(path, f), "r")))
else:
    data = json.load(open(path, "r"))
print("{} samples found".format(len(data)))

html = "<html><body>"
html += "<style>th {position: sticky; top: 0;background: FloralWhite;}</style>"
html += '<table border="0" style="table-layout: fixed; width: 100%; word-break:break-word">'
html += '<tr bgcolor=gray><th width=5%>Index</th><th width=25%>Q & Pos Snippets</th><th width=40%>Neg Snippets</th><th width=30%>X_modal Facts</th></tr>'
count = 0
for k in data: #[34, 279, 450, 474, 563, 613, 712, 842, 1311, 1793, 1936, 2340, 4266, 5279, 5620, 5800, 6845, 7018, 7529]:
    word_lists = [l.split(" || ") for l in list(data[k]['word_lists'].values())]
    sen2score = {}
    for f in data[k]['new_negFacts']:
        sen2score[f['fact']] = {
            'title': f['title'],
            'scores': tuple(float(x) for x in f['scores'][1:-1].split(",")),
            'link': f['url']
        }
    cap2score = {}
    for f in data[k]['img_negFacts']:
        cap2score[f['caption']] = {
            'title': f['title'],
            'scores': tuple(float(x) for x in f['scores'][1:-1].split(",")),
            'link': f['url'],
            'img': f['imgUrl']
        }
    html += add_html_row(k, sen2score, cap2score, word_lists)
    o = open('x_distractor_for_txt_demo2.html', 'wt')

    o.write(html)
    o.close()
html += '</table></body></html>'
o = open('x_distractor_for_txt_demo2.html', 'wt')

o.write(html)
o.close()

4 files found
60 samples found


In [388]:
### Mining + Create demo

html = "<html><body>"
html += "<style>th {position: sticky; top: 0;background: FloralWhite;}</style>"
html += '<table border="0" style="table-layout: fixed; width: 100%; word-break:break-word">'
html += '<tr bgcolor=gray><th width=5%>Index</th><th width=25%>Q & Pos Snippets</th><th width=40%>Neg Snippets</th><th width=30%>X_modal Facts</th></tr>'
count = 0
for k in [3787]: #[34, 279, 450, 474, 563, 613, 712, 842, 1311, 1793, 1936, 2340, 4266, 5279, 5620, 5800, 6845, 7018, 7529]:
    count += 1
    sen2score, cap2score, word_lists = get_sen2score_from_indx(k)
    html += add_html_row(k, sen2score, cap2score, word_lists)
    o = open('x_distractor_for_txt_demo3.html', 'wt')

    o.write(html)
    o.close()
html += '</table></body></html>'
o = open('x_distractor_for_txt_demo3.html', 'wt')

o.write(html)
o.close()

k =  3787
#pages before extension by hyperlink:  18
#pages after extension by hyperlink:  42
Q =  Which rival groups did the people who are currently the dominant ethnic group in the Xinjiang region compete with in the past?
A =  They competed with other Altaic tribes, Indo-European empires from the south and west and Sino-Tibetan empires to the east.
keywords =  {'dominant', 'ethnic', 'rival', 'Region', 'region', 'xinjiang', 'Past', 'Rival', 'Xinjiang', 'groups', 'group', 'Group', 'People', 'people', 'Dominant', 'Groups', 'Ethnic', 'past'}
titlewords =  {'History', 'Xinjiang', 'Christianity', 'people', 'Uyghur'}
answerwords =  {'tribes', 'indo', 'south', 'West', 'Sino', 'Tibetan', 'european', 'Other', 'Empires', 'Altaic', 'other', 'Indo', 'South', 'tibetan', 'east', 'European', 'East', 'empires', 'sino', 'Tribes', 'west', 'altaic'}
goldfactwords =  {'Brief', 'Republic', 'powers', 'story', 'Christianity', 'muslim', 'Tribe', 'few', 'Mountains', 'Nomadic', 'nomadic', 'small', 'Story', 'r

In [373]:
### no https: before '//upload'
with open('0719demo.html','r') as f:
    file = f.read()
f.close()
print(type(file))
print(len(file))
indices = [i for i in range(len(file)) if file.startswith('//upload.wikimedia.org/wikipedia/commons/', i)]
count = 0
for i in indices:
    file = file[:i+6*count] + 'https:' + file[6*count+i:]
    count += 1
print(len(file))
o = open('0719_demo.html', 'wt')

o.write(file)
o.close()

<class 'str'>


In [344]:
count = 0
for k in new_txt_data:
    if all(['en.wikipedia.org/wiki/' not in f['url'] for f in new_txt_data[k]['SupportingFacts']]):
        count += 1
print(count)

248


In [260]:
titlewords, keywords, goldfactwords, answerwords, Q, A, anchor2page = get_keywords_and_relevant_pages_from_txt_sample("3435")

#pages before extension by hyperlink:  20
#pages after extension by hyperlink:  40


In [318]:
for k in new_txt_data:
    Q = new_txt_data[k]['Q']
    if ' or ' in Q:
        if random.random() < 0.05:
            print(k)
            print(Q)

34
Who lived a longer life: Henry I, Count of Anhalt or Louis IV, Landgrave of Thuringia?
279
Were the attackers during the Battle of Malaga and the Catalonia Offensive of the Spanish Civil War Nationalist or Republican?
349
Did the Russian Revolution conclude before or after the Russian Famine began?
361
Did United Airlines Flight 811 and United Airlines Flight 553 crash in the same year or different years?
448
Was the album Goodnight Moon released before or after the book of the same name was published?
450
Which organization had more countries as members: the Arab League or the Entente Powers?
474
Has Yeshimon and the Judaean desert been urbanized or unaltered by human activity?
514
Is the abdominal area or the lower torso closer to the medial compartment that contains the gracilis muscle?
563
Who has been more prolific and varied in their career: Joel Coen or Anthony Cotton?
613
Which took place later in the 2017 calendar year: The attack on Saint Menas church or the Palm Sunday ch

In [320]:
all([b in "in the same or different years" for b in [' same ', ' or ', ' different ']])

True

In [299]:
for s in sen2score:
    if 'This feature is standardized on CD-R' in s:
        print(s)
        print('')
        print(sen2score[s])

This feature is standardized on CD-R, CD-RW, DVD-R and DVD-RW, but not on DVD+R and DVD+RW, on which only Plextor optical drives support simulated writing so far.

{'scores': (0.15, 0.0, 0.0, 0.15, 0.15), 'link': 'https://en.wikipedia.org/wiki/Optical_disc_recording_technologies', 'title': 'Optical disc recording technologies'}


In [302]:
s = nlp('This feature is standardized on CD-R, CD-RW, DVD-R and DVD-RW, but not on DVD+R and DVD+RW, on which only Plextor optical drives support simulated writing so far.')
[(t.pos_, t.text) for t in s]

[('DET', 'This'),
 ('NOUN', 'feature'),
 ('AUX', 'is'),
 ('VERB', 'standardized'),
 ('ADP', 'on'),
 ('PROPN', 'CD'),
 ('PUNCT', '-'),
 ('NOUN', 'R'),
 ('PUNCT', ','),
 ('PROPN', 'CD'),
 ('PUNCT', '-'),
 ('NOUN', 'RW'),
 ('PUNCT', ','),
 ('NOUN', 'DVD'),
 ('PUNCT', '-'),
 ('NOUN', 'R'),
 ('CCONJ', 'and'),
 ('NOUN', 'DVD'),
 ('PUNCT', '-'),
 ('PROPN', 'RW'),
 ('PUNCT', ','),
 ('CCONJ', 'but'),
 ('PART', 'not'),
 ('ADP', 'on'),
 ('PROPN', 'DVD+R'),
 ('CCONJ', 'and'),
 ('PROPN', 'DVD+RW'),
 ('PUNCT', ','),
 ('ADP', 'on'),
 ('DET', 'which'),
 ('ADV', 'only'),
 ('PROPN', 'Plextor'),
 ('ADJ', 'optical'),
 ('NOUN', 'drives'),
 ('VERB', 'support'),
 ('VERB', 'simulated'),
 ('NOUN', 'writing'),
 ('ADV', 'so'),
 ('ADV', 'far'),
 ('PUNCT', '.')]

In [289]:
print(' ------------ snippets -------------')
for k in list(sen2score.keys())[:10]:
    print(k, sen2score[k])
print(' ------------ imgs ------------- ')
for k in list(cap2score.keys())[:10]:
    print(k, cap2score[k])

 ------------ snippets -------------
Kramer's physical format is used in all optical discs. {'scores': (0.29, 0.0, 0.0, 0.29, 0.29), 'link': 'https://en.wikipedia.org/wiki/Optical_discs', 'title': 'Optical disc'}
The discs are then silkscreened or a label is otherwise applied. {'scores': (0.2, 0.0, 0.0, 0.2, 0.2), 'link': 'https://en.wikipedia.org/wiki/Optical_discs', 'title': 'Optical disc'}
Most optical discs exhibit a characteristic iridescence as a result of the diffraction grating formed by its grooves. {'scores': (0.18, 0.0, 0.0, 0.18, 0.18), 'link': 'https://en.wikipedia.org/wiki/Optical_discs', 'title': 'Optical disc'}
For computer data backup and physical data transfer, optical discs such as CDs and DVDs are gradually being replaced with faster, smaller solid-state devices, especially the USB flash drive. This trend is expected to continue as USB flash drives continue to increase in capacity and drop in price. {'scores': (0.18, 0.0, 0.01, 0.18, 0.16999999999999998), 'link': 'h

In [288]:
cap2score = dict(sorted(cap2score.items(), key=lambda x: x[1]['scores'][-1], reverse=True))
list(cap2score.keys())

[' Minidiscs are magneto-optical discs used to store music.',
 'Optical discs',
 ' Assortment of flash drives',
 ' The optical lens of a compact disc drive.',
 ' A Magneto-optical disc and the sector partition rectangles on its surface.',
 ' Two 2.5" external USB hard drives',
 ' A 130 mm 2.6GB magneto-optical disc.',
 ' A 230 MB Fujitsu 90 mm magneto-optical disc.',
 ' From left to right: full-height 5.25″ drive, two half-height 5.25″ drives, and (sideways) a 3.5″ drive']